In [1]:
import pandas as pd

In [71]:
df1 = pd.read_excel("Training.xlsx")
df2 = pd.read_excel("Validation.xlsx")

In [72]:
df1.drop(["연령", "성별", "신체질환", "상황키워드", "감정_소분류",  "시스템문장1", "시스템문장2", "시스템문장3"], axis=1, inplace=True)
df2.drop(["연령", "성별", "신체질환", "상황키워드", "감정_소분류",  "시스템문장1", "시스템문장2", "시스템문장3"], axis=1, inplace=True)

In [79]:
df1.rename(columns={"사람문장1" : "sentence"}, inplace=True)
df1.rename(columns={'감정_대분류': 'Emotion'}, inplace=True)

df2.rename(columns={"사람문장1" : "sentence"}, inplace=True)
df2.rename(columns={'감정_대분류': 'Emotion'}, inplace=True)


In [80]:
df_train = pd.concat([
    df1[['Emotion', 'sentence']],
    df1[['Emotion', '사람문장2']].rename(columns={'사람문장2': 'sentence'}),
    df1[['Emotion', '사람문장3']].rename(columns={'사람문장3': 'sentence'})
], ignore_index=True)

df_valid = pd.concat([
    df2[['Emotion', 'sentence']],
    df2[['Emotion', '사람문장2']].rename(columns={'사람문장2': 'sentence'}),
    df2[['Emotion', '사람문장3']].rename(columns={'사람문장3': 'sentence'})
], ignore_index=True)


In [84]:
df_train = df_train.dropna()
df_valid = df_valid.dropna()

In [88]:
df_train.to_csv("training_data.csv", index=False)
df_valid.to_csv("validation_data.csv", index=False)

In [36]:
import pandas as pd

In [37]:
df_train = pd.read_csv("training_data.csv")
df_valid = pd.read_csv("validation_data.csv")

In [38]:
from konlpy.tag import Okt

In [47]:
train = pd.read_csv("training_data.csv")

In [48]:
okt = Okt()

temp = []
for i in train['sentence']:
    temp.append(okt.morphs(i))
    
temp = pd.Series(temp)
train['token'] = temp

In [49]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145955 entries, 0 to 145954
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Emotion   145955 non-null  object
 1   sentence  145955 non-null  object
 2   token     145955 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB


In [50]:
train.to_csv("training_data-token.csv", index=False)

In [51]:
import pandas as pd

In [52]:
df_train = pd.read_csv("training_data-token.csv")

In [56]:
df_train['token']

0         ['일', '은', '왜', '해도', '해도', '끝', '이', '없을까', '...
1         ['이번', '달', '에', '또', '급여', '가', '깎였어', '!', '...
2         ['회사', '에', '신입', '이', '들어왔는데', '말투', '가', '거슬...
3         ['직장', '에서', '막내', '라는', '이유', '로', '나', '에게만'...
4         ['얼마', '전', '입사', '한', '신입사원', '이', '나를', '무시'...
                                ...                        
145950    ['노년', '층', '을', '위', '한', '경제', '적', '지원', '이...
145951    ['남편', '과', '함께', '게이트볼', '이나', '치러', '가야겠어', ...
145952    ['남편', '과', '함께', '실버', '일자리', '나', '노년', '층',...
145953    ['함께', '친하게', '지내던', '동네', '언니', '동생', '들', '과...
145954    ['사람', '들', '을', '볼', '때', '의심', '하고', '불신', '...
Name: token, Length: 145955, dtype: object

In [73]:
from gensim.models import Word2Vec
model = Word2Vec(df_train['token'], vector_size=1000, window=5, min_count=1, workers=4, sg=1)

In [74]:
model.wv.vectors.shape

(1468, 1000)

In [77]:
import joblib

joblib.dump(model, 'word_model.joblib')

['word_model.joblib']

In [ ]:
import pandas as pd
import numpy as np

In [2]:
def get_sent_embedding(model, embedding_size, tokenized_words):
    feature_vec = np.zeros((embedding_size,), dtype="float32")
    n_words = 0
    for word in tokenized_words:
        if word in model.wv.key_to_index:
            n_words += 1
            feature_vec = np.add(feature_vec, model.wv[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [3]:
df_train = pd.read_csv("training_data-token.csv")

: 

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from joblib import load, dump
import numpy as np

# Load word_model.joblib
model = load("word_model.joblib")

# Prepare data
X_train = np.array([get_sent_embedding(model, 1000, tokens) for tokens in df_train['token']])
y_train = df_train['Emotion']

# Convert labels to one-hot encoding
num_classes = len(np.unique(y_train))
y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes)

# Convert data to TensorFlow tensors
X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train_one_hot, dtype=tf.float32)

# Define SVM model using TensorFlow
svm_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),  # Input layer
    tf.keras.layers.Dense(units=num_classes, activation='softmax')  # SVM with softmax activation for multiclass classification
])

# Compile the model
svm_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
svm_model.fit(X_train_tf, y_train_tf, epochs=100, batch_size=32)

# Save the trained model
svm_model.save("svm_model_tensorflow")

dump(svm_model, 'svm_model.joblib')

In [4]:
from sklearn.svm import SVC
from joblib import load, dump
model = load("word_model.joblib")

X = [get_sent_embedding(model, 1000, tokens) for tokens in df_train['token']]
y = df_train['Emotion']

# SVM 분류 모델 학습
svm_model = SVC(kernel='linear')
svm_model.fit(X, y)
dump(svm_model, 'svm_model.joblib')

In [ ]:
from konlpy.tag import Okt
from gensim.models import Word2Vec
from sklearn.svm import SVC

# Okt 형태소 분석기 초기화
okt = Okt()

# 형태소 분석 함수 정의
def tokenize(text):
    return okt.morphs(text)

# 주어진 문장
sentence = "이 영화 정말 재미있었어!"

# 형태소로 분할
tokenized_sentence = tokenize(sentence)

# Word2Vec 모델 불러오기
model = Word2Vec.load("word2vec.model")

# 형태소를 Word2Vec 임베딩으로 변환하는 함수 정의
def get_embedding(model, tokens):
    embeddings = [model.wv[token] for token in tokens if token in model.wv.key_to_index]
    if embeddings:
        return sum(embeddings) / len(embeddings)
    else:
        return []

# 문장을 Word2Vec 임베딩으로 변환
embedding = get_embedding(model, tokenized_sentence)

# SVM 모델 불러오기
svm_model = SVC(kernel='linear')
svm_model.load("svm_model.joblib")

# 감정 분류
emotion = svm_model.predict([embedding])[0]

print("입력 문장:", sentence)
print("예측된 감정:", emotion)
